In [4]:
import pandas as pd
import cv2
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import random
import sys
import scipy

In [5]:
class Img():

  def __init__(self, path="", matrix=None):

    if matrix is None:
      self.img = np.array(cv2.cvtColor(cv2.imread(f"imgs/{path}"), cv2.COLOR_BGR2GRAY), dtype=np.float64)
    else:
      self.img = matrix

    self.autovals = []
    self.gauss_img = np.zeros((self.img.shape[0], self.img.shape[1]), dtype=np.float64)
    self.salt_img = np.array(self.img)
    self.U = []
    self.D = []
    self.Vt = []

  def show(self, title=None):
    plt.imshow(self.img, cmap='gray')
    plt.title('Imagem') if title is None else plt.title(title)
    plt.show()

  def svd_numpy(self):
    self.U, self.D, self.Vt = np.linalg.svd(self.img, full_matrices=False)

  def reconstruct_numpy(self, rank, show=False):
    if len(self.U) + len(self.D) + len(self.Vt) == 0:
      self.svd_numpy()

    recon = np.dot(self.U[:, :rank], np.dot(np.diag(self.D[:rank]), self.Vt[:rank, :]))
    if show:
      plt.imshow(recon, cmap='gray')
      plt.title('Imagem reconstruída')
      plt.show()
    return Img(matrix=recon)

  def svd_scipy(self, rank):
    self.U, self.D, self.Vt = scipy.sparse.linalg.svds(self.img, k=rank)

  def reconstruct_scipy(self, rank=None, show=False):
    if len(self.U) + len(self.D) + len(self.Vt) == 0:
      self.svd_scipy(rank)

    recon = np.dot(self.U, np.dot(np.diag(self.D), self.Vt))
    if show:
      plt.imshow(recon, cmap='gray')
      plt.title('Imagem reconstruída')
      plt.show()
    return Img(matrix=recon)

  def ruido_salt_n_pepper(self, taxa):

    """ Gera uma matriz de rand nas dimensoes da imagem e soma aos valores da representação matricial da imagem """

    altura, largura = self.img.shape
    nS_pixels =  random.randint(100, 100+taxa)
    nP_pixels =  random.randint(100, 100+taxa)

    for i in range(nS_pixels):
      y = random.randint(0, altura - 1)
      x = random.randint(0, largura - 1)

      self.salt_img[y][x] = 255

    for i in range(nP_pixels):
      y = random.randint(0, altura - 1)
      x = random.randint(0, largura - 1)
      self.salt_img[y][x] = 0

    return Img(matrix=self.salt_img)


  def ruido_gaussiano(self, taxa):

    """ Gera uma matriz de rand nas dimensoes da self e soma aos valores da representação matricial da self. """

    altura, largura = self.img.shape
    ruido =  np.random.rand(altura, largura) * taxa

    self.gauss_img = ruido + self.img

    return Img(matrix=self.gauss_img)

  def plot_auto_val(self, rank = True):
    if self.D == []:
      self.svd()

    if rank == True:
        rank = self.D.shape[0]
    self.autovals = [self.D[i] for i in range(rank)]

    fig = plt.figure()
    ax = fig.gca()
    ax.plot(self.autovals)
    plt.yscale('log')
    #ax.plot(10, self.autovals[10], 'ro')
    plt.show()

In [23]:
madonna = Img(path="madonna.png")

madonna_scipy  = madonna.reconstruct_scipy(80)
print("Dimensão da imagem original ", np.shape(madonna.img))
print("Memória usada originalmente (em bytes): ", madonna.img.nbytes)

print("\nDimensões da matriz U: ", np.shape(madonna.U))
print("Memória gasta pela matriz U (em bytes): ", np.array(madonna.U).nbytes)

print("\nDimensões da matriz D: ", np.shape(madonna.D))
print("Memória gasta pela matriz D (em bytes): ", np.array(madonna.D).nbytes)

print("\nDimensões da matriz Vt: ", np.shape(madonna.Vt))
print("Memória gasta pela matriz Vt (em bytes): ", np.array(madonna.Vt).nbytes)

soma = np.array(madonna.U).nbytes + np.array(madonna.D).nbytes + np.array(madonna.Vt).nbytes

print("\nSoma da memória gasta pelas partes (em bytes)", soma)



Dimensão da imagem original:  (984, 984)
Memória usada originalmente (em bytes):  7746048

Dimensões da matriz U:  (984, 80)
Memória gasta pela matriz U (em bytes):  629760

Dimensões da matriz D:  (80,)
Memória gasta pela matriz D (em bytes):  640

Dimensões da matriz Vt:  (80, 984)
Memória gasta pela matriz Vt (em bytes):  629760

Soma da memória gasta pelas partes (em bytes):  1260160


In [4]:
franca = Img(path="franca.png")

print("imagem inteira: ", franca.img.nbytes)

franca_recon = franca.reconstruct_2(1)
print("U: ", np.array(franca.U).nbytes, np.shape(franca.U))
print("D: ", np.array(franca.D).nbytes, np.shape(franca.D))
print("Vt: ", np.array(franca.Vt).nbytes, np.shape(franca.Vt))
print("imagem intereconira: ", franca.img.nbytes)
print("soma: ", np.array(franca.U).nbytes  +  np.array(franca.D).nbytes +  np.array(franca.Vt).nbytes)

imagem inteira:  7680000
U:  6400 (800, 1)
D:  8 (1,)
Vt:  9600 (1, 1200)
imagem intereconira:  7680000
soma:  16008
